## MMA Summer Datathon

### 1. Load the CSV file

In [2]:
import pandas as pd

hockey_data = pd.read_csv('Rotman MMA Summer Datathon NWHL.csv')

In [3]:
hockey_data.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2021-01-23,Minnesota Whitecaps,Boston Pride,1,20:00,5,5,0,0,Boston Pride,...,Faceoff Win,100,43,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN
1,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:58,5,5,0,0,Boston Pride,...,Puck Recovery,107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:57,5,5,0,0,Boston Pride,...,Zone Entry,125,28,Carried,NaN,NaN,NaN,Maddie Rowe,NaN,NaN
3,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:55,5,5,0,0,Boston Pride,...,Shot,131,28,Snapshot,On Net,t,f,NaN,NaN,NaN
4,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:53,5,5,0,0,Boston Pride,...,Faceoff Win,169,21,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN


In [4]:
players=hockey_data.Player.unique()

### 2. Analysis for best Goal Scorer

##### Metrics Considered:
a. Average number of goals per game -> No. of goals/ No. of games <br>
b. Success Rate -> No. of goals/ (No. of goals + No. of shots)<br>
<br> _How are we assigning weights ?_ 
<br> <br>
c. Distance or Position of Scorer (xG) - To be completed<br>

In [123]:
best_goal=[]

for player in players:
    gc=0
    sc=0
    nog=0
    gamedate=[]
    for index, row in hockey_data.iterrows():
        if row['Player']==player:
            if row['Event']=='Goal':
                gc=gc+1
            if row['Event']=='Shot':
                sc=sc+1
            if row['game_date'] not in gamedate:
                nog=nog+1
                gamedate.append(row['game_date'])
    #print("Total goals for " + player + " is "+ str(gc) + " and shots is " + str(sc) + " games played = " + str(nog))
    if gc!=0:#------------------------>Handle Division by Zero
        ag=gc/nog
        sr=gc/(gc+sc)
    else:
        ag=0
        sr=0
    best_goal.append([player,gc,sc,nog,ag,sr])
bgdf=pd.DataFrame(best_goal,columns=['Name','Goals','Shots','Games played','Average Goals per Game','Success Rate'])
print("Done!!!") #-------------> Name, goal count, shot count, games played, gc/nog, gc/gc+sc Stored in DataFrame

Done!!!


In [133]:
bgdf['Average Weighted Value']=bgdf[['Average Goals per Game','Success Rate']].mean(axis=1)
bgdf=bgdf.sort_values('Average Weighted Value', ascending=False)
bgdf=bgdf.reset_index(drop=True)
bgdf.to_csv('Best_Goal_Data.csv') #--------------> Saving the df to CSV
bgdf.head(10)

,Name,Goals,Shots,Games played,Average Goals per Game,Success Rate,Average Weighted Value
0,Leila Kilduff,2,5,3,0.666667,0.285714,0.476190
1,Mikyla Grant-Mentis,5,49,6,0.833333,0.092593,0.462963
2,Taylor Woods,4,42,6,0.666667,0.086957,0.376812
3,Samantha Davis,4,34,7,0.571429,0.105263,0.338346
4,Autumn MacDougall,3,18,6,0.500000,0.142857,0.321429
5,Haley Mack,2,13,4,0.500000,0.133333,0.316667
6,Audra Richards,2,14,4,0.500000,0.125000,0.312500
7,Jillian Dempsey,3,24,6,0.500000,0.111111,0.305556
8,Nina Rodgers,2,17,4,0.500000,0.105263,0.302632
9,Katelynn Russ,2,26,4,0.500000,0.071429,0.285714


# Metric : xG
a. Distance <br>
b. Type

In [22]:
#Based on Just Distance

import math

xG=[]
for index,row in hockey_data.iterrows():
    if row['Event']=='Shot' or row['Event']=='Goal':
        if row['Detail 2'] !='Blocked':
            dist=math.sqrt(((200-row['X Coordinate'])**2)+((42.5-row['Y Coordinate'])**2))
            if row['Event']=='Goal':
                xG.append([row['Player'],row['Event'],row['X Coordinate'],row['Y Coordinate'],dist,row['Detail 1'],row['Detail 2'],1])
            if row['Event']=='Shot':
                xG.append([row['Player'],row['Event'],row['X Coordinate'],row['Y Coordinate'],dist,row['Detail 1'],row['Detail 2'],0])

xG_df=pd.DataFrame(xG,columns=['Name','Event','X','Y','Distance','Type','Detail','Result'])
print("Done!!!")


Done!!!


,Name,Event,X,Y,Distance,Type,Detail,Result
0,McKenna Brand,Shot,131,28,70.507092,Snapshot,On Net,0
1,Allie Thunstrom,Shot,162,2,55.536024,Snapshot,On Net,0
2,Nina Rodgers,Shot,167,18,41.100487,Snapshot,Missed,0
3,McKenna Brand,Shot,175,68,35.710643,Snapshot,On Net,0
4,Lauren Kelly,Shot,196,20,22.852790,Snapshot,On Net,0


In [43]:
# Binomial Regression Fitting for Distance only model

import statsmodels.api as sm
import statsmodels.formula.api as smf

model_variables = ['Distance']
model=''
for v in model_variables[:-1]:
    model = model  + v + ' + '
model = model + model_variables[-1]

#Fit the model
test_model = smf.glm(formula="Result ~ " + model, data=xG_df, 
                           family=sm.families.Binomial()).fit()
print(test_model.summary())        
b=test_model.params

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 Result   No. Observations:                 1420
Model:                            GLM   Df Residuals:                     1418
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -278.75
Date:                Sat, 05 Jun 2021   Deviance:                       557.51
Time:                        14:39:38   Pearson chi2:                 1.58e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3592      0.269     -5.047      0.0

In [45]:
def calculate_xG(sh):    # -> function to caluculate xG after training to each dataframe object
   bsum=b[0]
   for i,v in enumerate(model_variables):
       bsum=bsum+b[i+1]*sh[v]
   xG = 1/(1+np.exp(bsum)) 
   return xG   

In [46]:
#Call for calculate-xG function above

xG_new=xG_df
xG_new.head(5)
xG_val=xG_new.apply(calculate_xG, axis=1) 
xG_new = xG_new.assign(xG_val=xG_val)

In [48]:
xG_new.head(25)


,Name,Event,X,Y,Distance,Type,Detail,Result,xG_val
0,McKenna Brand,Shot,131,28,70.507092,Snapshot,On Net,0,0.987331
1,Allie Thunstrom,Shot,162,2,55.536024,Snapshot,On Net,0,0.976328
2,Nina Rodgers,Shot,167,18,41.100487,Snapshot,Missed,0,0.957139
3,McKenna Brand,Shot,175,68,35.710643,Snapshot,On Net,0,0.946693
4,Lauren Kelly,Shot,196,20,22.852790,Snapshot,On Net,0,0.911367
5,Haylea Schmid,Shot,181,43,19.006578,Snapshot,On Net,0,0.897244
6,Lauren Kelly,Shot,163,10,49.246827,Snapshot,On Net,0,0.969297
7,Samantha Davis,Shot,188,49,13.647344,Wrap Around,On Net,0,0.874264
8,Jillian Dempsey,Shot,149,19,56.153807,Snapshot,On Net,0,0.976928
9,Jonna Curtis,Shot,167,67,41.100487,Snapshot,On Net,0,0.957139


In [58]:
# xG calculation considering both Type of Shot and Distance ->Modify This

type_shot=0 #-----------------------> 1.Deflection 2.Snapshot 3.Slapshot 4.Wristshot 5.Fan 6. Wrap Around
xG=[]
for index,row in hockey_data.iterrows():
    if row['Event']=='Shot' or row['Event']=='Goal':
        if row['Detail 2'] !='Blocked':
            if row['Detail 1'] =='Deflection':
                type_shot=1
            if row['Detail 1'] =='Snapshot':
                type_shot=2
            if row['Detail 1'] =='Slapshot':
                type_shot=3
            if row['Detail 1'] =='Wristshot':
                type_shot=4
            if row['Detail 1'] =='Fan':
                type_shot=5
            if row['Detail 1'] =='Wrap Around':
                type_shot=6
                    
            dist=math.sqrt(((200-row['X Coordinate'])**2)+((42.5-row['Y Coordinate'])**2))
            if row['Event']=='Goal':
                xG.append([row['Player'],row['Event'],row['X Coordinate'],row['Y Coordinate'],dist,type_shot,row['Detail 1'],row['Detail 2'],1])
            if row['Event']=='Shot':
                xG.append([row['Player'],row['Event'],row['X Coordinate'],row['Y Coordinate'],dist,type_shot,row['Detail 1'],row['Detail 2'],0])

xG_df2=pd.DataFrame(xG,columns=['Name','Event','X','Y','Distance','ShotType','Type','Detail','Result'])
print("Done!!!")
xG_df2.head(5)

Done!!!


,Name,Event,X,Y,Distance,ShotType,Type,Detail,Result
0,McKenna Brand,Shot,131,28,70.507092,2,Snapshot,On Net,0
1,Allie Thunstrom,Shot,162,2,55.536024,2,Snapshot,On Net,0
2,Nina Rodgers,Shot,167,18,41.100487,2,Snapshot,Missed,0
3,McKenna Brand,Shot,175,68,35.710643,2,Snapshot,On Net,0
4,Lauren Kelly,Shot,196,20,22.852790,2,Snapshot,On Net,0


In [60]:
model_variables = ['Distance','ShotType']
model=''
for v in model_variables[:-1]:
    model = model  + v + ' + '
model = model + model_variables[-1]

#Fit the model
test_model = smf.glm(formula="Result ~ " + model, data=xG_df2, 
                           family=sm.families.Binomial()).fit()
print(test_model.summary())        
b=test_model.params

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 Result   No. Observations:                 1420
Model:                            GLM   Df Residuals:                     1417
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -277.43
Date:                Sat, 05 Jun 2021   Deviance:                       554.87
Time:                        15:06:42   Pearson chi2:                 1.59e+03
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9561      0.362     -2.640      0.0

In [61]:
xG_new2=xG_df2
xG_new2.head(5)
xG_val=xG_new2.apply(calculate_xG, axis=1) 
xG_new2 = xG_new2.assign(xG_val=xG_val)

In [62]:
xG_new2.head(25)

,Name,Event,X,Y,Distance,ShotType,Type,Detail,Result,xG_val
0,McKenna Brand,Shot,131,28,70.507092,2,Snapshot,On Net,0,0.984756
1,Allie Thunstrom,Shot,162,2,55.536024,2,Snapshot,On Net,0,0.972237
2,Nina Rodgers,Shot,167,18,41.100487,2,Snapshot,Missed,0,0.950991
3,McKenna Brand,Shot,175,68,35.710643,2,Snapshot,On Net,0,0.939634
4,Lauren Kelly,Shot,196,20,22.852790,2,Snapshot,On Net,0,0.901960
5,Haylea Schmid,Shot,181,43,19.006578,2,Snapshot,On Net,0,0.887144
6,Lauren Kelly,Shot,163,10,49.246827,2,Snapshot,On Net,0,0.964383
7,Samantha Davis,Shot,188,49,13.647344,6,Wrap Around,On Net,0,0.924115
8,Jillian Dempsey,Shot,149,19,56.153807,2,Snapshot,On Net,0,0.972911
9,Jonna Curtis,Shot,167,67,41.100487,2,Snapshot,On Net,0,0.950991


In [63]:
xG_new2.to_csv('xG_dist+type.csv') #----> CSV with xG for each shot

In [66]:
avg_xG=[]

for player in players:
    avg=0
    s=0
    tot=0
    exp=0
    for index, row in xG_new2.iterrows():
        if row['Name']==player:
            tot=tot+1
            s=row['xG_val']+s
            if row['Result']==1:
                exp=exp+1
    if tot !=0:
        avg=s/tot
        avg_xG.append([player,avg,exp])
xGpl=pd.DataFrame(avg_xG,columns=['Name','AvgXG','Goals'])
print("Done!!!") 
xGpl.to_csv('xG_avg-player.csv') # ---------------------------> Average xG for each Player along with number of goals

Done!!!


https://public.tableau.com/views/xGvsNo_ofGoals/Sheet1?:language=en-GB&:display_count=n&:origin=viz_share_link  <br>

Above link for xG vs No of goals for each player 